In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import differential_evolution

In [2]:
#read data
d = pd.read_csv('C:/Users/cape159/Documents/pracovni/data_statistika/kopackuv_grant/mixing_experiment/DB_concept/Hasan_Jolanta/hasan_jolanta.csv')

print(d)

   Soil Status   time   G12  G12init  G13  G13init  CO212  CO213      DOC12  \
0    PL      A    0.1  72.3     97.7  2.0      2.7    0.7    0.0        NaN   
1    PL      A   12.0  14.0     97.7  0.4      2.7   22.1    0.5        NaN   
2    PL      A   24.0   0.3     97.7  0.0      2.7   36.1    0.9  21.204962   
3    PL      A   48.0   0.2     97.7  0.0      2.7   50.3    1.1  19.468059   
4    PL      A   72.0   0.0     97.7  0.0      2.7   61.3    1.3  19.662000   
5    CT      A    0.1  64.0     89.0  1.8      2.5    0.5    0.0        NaN   
6    CT      A   12.0  29.7     89.0  0.8      2.5   10.3    0.2        NaN   
7    CT      A   24.0   5.3     89.0  0.1      2.5   20.8    0.5  21.628389   
8    CT      A   48.0   0.2     89.0  0.0      2.5   37.9    0.9  18.054769   
9    CT      A   72.0   0.0     89.0  0.0      2.5   44.5    1.1  17.284000   
10   PL      N    0.1  78.0    105.3  2.1      2.9    0.2    0.0        NaN   
11   PL      N   12.0  74.4    105.3  2.0      2.9  

In [3]:
#define DB model
def DBmodel (y, t, pars):
    #define initial states
    R12=y[0];    S12=y[1];    G12=y[2];   DOC12=y[3];   CO212=y[4]
    R13=y[5];    S13=y[6];    G13=y[7];   DOC13=y[8];   CO213=y[9]
        
    #define parameters
    Acg=pars[0];   Vmaxg=pars[1];    Kmg=pars[2] 
    Ac=pars[3];    Vmax=pars[4];     Km=pars[5] 
    m0=pars[6];    f=pars[7];        Yu=pars[8]
    D=pars[9];     fs=pars[10]
        
    
    #G uptake rate total
    Cug_tot=Vmaxg*(S12+S13)*(G12+G13)/(G12+G13+Kmg)
        
    #DOC uptake rate total
    Cu_tot=Vmax*(S12+S13)*(DOC12+DOC13)/(DOC12+DOC13+Km)
        
    #maintnance rate total
    m_tot=m0*(S12+S13)
            
    #reserves mobilization rate total
    an_tot=f*(R12+R13)-m_tot
        
    #define 13C atm% scaling factors
    Ratm=R13/(R12+R13)
    Satm=S13/(S12+S13)
    Gatm=G13/(G12+G13)-D
    DOCatm=DOC13/(DOC12+DOC13)
    
    #respiration rate 12 C
    if an_tot > 0:
        r12=m_tot*(1-Ratm)+an_tot*(1-Yu)*(1-Ratm)+(1-Acg)*Cug_tot*(1-Gatm)+(1-Ac)*Cu_tot*(1-DOCatm)
    else:
        r12=f*R12+(1-Acg)*Cug_tot*(1-Gatm)+(1-Ac)*Cu_tot*(1-DOCatm)
             
    #respiration rate 13 C
    if an_tot > 0:
        r13=m_tot*Ratm+an_tot*(1-Yu)*Ratm+(1-Acg)*Cug_tot*Gatm+(1-Ac)*Cu_tot*DOCatm
    else:
        r13=f*R13+(1-Acg)*Cug_tot*Gatm+(1-Ac)*Cu_tot*DOCatm
                  
            
    #derivatives
    dR12dt=Acg*Cug_tot*(1-Gatm)+Ac*Cu_tot*(1-DOCatm)-f*R12
    dS12dt=np.maximum(an_tot*Yu*(1-Ratm), 0)+np.minimum(0, an_tot/m0*(1-Satm))
    dG12dt=-Cug_tot*(1-Gatm)
    dDOC12dt=-Cu_tot*(1-DOCatm)-np.minimum(0, an_tot/m0*(1-Satm)*fs)
    dCO212dt=r12
    dR13dt=Acg*Cug_tot*Gatm+Ac*Cu_tot*DOCatm-f*R13
    dS13dt=np.maximum(an_tot*Yu*Ratm, 0)+np.minimum(0, an_tot/m0*Satm)
    dG13dt=-Cug_tot*Gatm
    dDOC13dt=-Cu_tot*DOCatm-np.minimum(0, an_tot/m0*Satm*fs)
    dCO213dt=r13
    
            
    return dR12dt, dS12dt, dG12dt, dDOC12dt, dCO212dt, dR13dt, dS13dt, dG13dt, dDOC13dt, dCO213dt;

In [4]:
#define a function returning ode results with additional calculations
def calc (model, pars, t, y0):
    #these are the model parameters
    pars1=pars[0:11]
    
    #these are the parameters to recalculate R and S to Cmic
    pars2=pars[10:12]
    
    #first solve the model
    y = odeint(model,y0,t, args=(pars1,))
    #y = pd.DataFrame(y)
    #y.columns = ['R', 'S', 'DOC', 'CO2']
    #Cu=pars1[1]*y[:, 1]*y[:, 2]/(y[:, 2]+pars1[2])
    
    #calculate respiration rates and add it to y frame
    #r = y[:, 1]*pars1[3]+np.maximum((pars1[4]*y[:, 0]-y[:, 1]*pars1[3])*(1-pars1[5]), 0)+Cu*(1-pars1[0])
    
    #calculate Cmic12 and add it to y frame
    Cmic12 = pars2[1] * y[:, 0] + pars2[0] * y[:, 1]
    #calculate Cmic13 and add it to y frame
    Cmic13 = pars2[1] * y[:, 5] + pars2[0] * y[:, 6]
    
    yhat = np.concatenate((y[:, 2].reshape(5,1),#G12
                           y[:, 3].reshape(5,1),#DOC12
                           y[:, 4].reshape(5,1),#CO212
                           y[:, 7].reshape(5,1),#G13
                           y[:, 8].reshape(5,1),#DOC13
                           y[:, 9].reshape(5,1),#CO213
                           Cmic12.reshape(5,1),
                           Cmic13.reshape(5,1)), axis=1)
    
    return yhat

In [5]:
#create the minimization function
def obj_fun (x):
    #define parameters
    pars = x
    
    #initial conditions
    Cmic12init = data.Cmic12init[0]
    G12init = data.G12init[0]
    DOC12init = data.DOC12init[0]
    Cmic13init = data.Cmic13init[0]
    G13init = data.G13init[0]
    DOC13init = data.DOC13init[0]
    R12init = pars[12]*(1-data.Cmic13init[0]/(data.Cmic12init[0]+data.Cmic13init[0]))
    R13init = pars[12]*(data.Cmic13init[0]/(data.Cmic12init[0]+data.Cmic13init[0]))
    S12init = (Cmic12init-R12init*pars[11])/pars[10]
    S13init = (Cmic13init-R13init*pars[11])/pars[10]
    
    
    y0 = np.array([R12init, S12init, G12init, DOC12init,0,
                   R13init, S13init, G13init, DOC13init,0])
    
    #times
    t = data.time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(DBmodel, pars[0:12], t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.G12]).reshape(5,1),
                        np.array([data.DOC12]).reshape(5,1),
                        np.array([data.CO212]).reshape(5,1),
                        np.array([data.G13]).reshape(5,1),
                        np.array([data.DOC13]).reshape(5,1),
                        np.array([data.CO213]).reshape(5,1),
                        np.array([data.Cmic12]).reshape(5,1),
                        np.array([data.Cmic13]).reshape(5,1)), 
                     axis=1)
    
    #weights
    weights=np.concatenate((np.nanmean(data.G12).repeat(5).reshape(5,1),
                            np.nanmean(data.DOC12).repeat(5).reshape(5,1),
                            np.nanmean(data.CO212).repeat(5).reshape(5,1),
                            np.nanmean(data.G13).repeat(5).reshape(5,1),
                            np.nanmean(data.DOC13).repeat(5).reshape(5,1),
                            np.nanmean(data.CO213).repeat(5).reshape(5,1),
                            np.nanmean(data.Cmic12).repeat(5).reshape(5,1),
                            np.nanmean(data.Cmic13).repeat(5).reshape(5,1)), 
                       axis=1)
                
          
    out=np.nansum(((yhat_full-obs)/weights)**2)
          
    return out

In [6]:
#create goodness of fit function
def goodness (x):
    #define parameters
    pars = x
    
    #initial conditions
    Cmic12init = data.Cmic12init[0]
    G12init = data.G12init[0]
    DOC12init = data.DOC12init[0]
    Cmic13init = data.Cmic13init[0]
    G13init = data.G13init[0]
    DOC13init = data.DOC13init[0]
    R12init = pars[12]*(1-data.Cmic13init[0]/(data.Cmic12init[0]+data.Cmic13init[0]))
    R13init = pars[12]*(data.Cmic13init[0]/(data.Cmic12init[0]+data.Cmic13init[0]))
    S12init = (Cmic12init-R12init*pars[11])/pars[10]
    S13init = (Cmic13init-R13init*pars[11])/pars[10]
    
    
    y0 = np.array([R12init, S12init, G12init, DOC12init,0,
                   R13init, S13init, G13init, DOC13init,0])
    
    #times
    t = data.time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(DBmodel, pars[0:12], t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.G12]).reshape(5,1),
                        np.array([data.DOC12]).reshape(5,1),
                        np.array([data.CO212]).reshape(5,1),
                        np.array([data.G13]).reshape(5,1),
                        np.array([data.DOC13]).reshape(5,1),
                        np.array([data.CO213]).reshape(5,1),
                        np.array([data.Cmic12]).reshape(5,1),
                        np.array([data.Cmic13]).reshape(5,1)), 
                     axis=1)                   
          
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = 2*13 - 2*ll
    
    out = np.array([R2, ll, AIC])
          
    return out

In [7]:
#Plesne aerobni
data = d[(d.Soil=='PL') & (d.Status=='A')]
data = data.reset_index(drop=True)
dataCmic = data.Cmic12init[0]

optimum_PA=differential_evolution(obj_fun, [(0, 1), (0.0001, 10), (0.1, 100), (0, 1), (0.0001, 10), (0.1, 100),
                                            (0.0001, 10), (0.0001, 10), (0, 1), 
                                            (1e-7, 1e-5), (0, 1), (0,1),
                                            (dataCmic*0.01, dataCmic*0.99)], 
                                  polish=True, maxiter=1000000)

print(optimum_PA)

C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


     fun: 4.605025744088067
     jac: array([ 8.24229573e-05,  6.37179198e-04,  3.49778873e-02, -4.58535432e-02,
       -1.96198613e-04, -1.40776280e-04, -9.98223726e-04, -8.48610071e-03,
       -8.42792502e-04, -1.12738613e+01,  4.27657909e-04, -9.12159237e-05,
       -4.40847359e-03])
 message: 'Optimization terminated successfully.'
    nfev: 40260
     nit: 199
 success: True
       x: array([7.90381125e-01, 2.33254291e+00, 1.00000000e-01, 1.00000000e+00,
       3.11360111e+00, 9.86921070e+01, 1.55162246e+00, 4.28082895e-03,
       7.02726359e-01, 1.00000000e-05, 9.58279018e-01, 4.41457422e-01,
       5.29894208e+01])


In [9]:
print(goodness(optimum_PA.x))
np.savetxt('PL_parametersD.csv', optimum_PA.x.reshape(1,13), delimiter=",")

[ 0.95537181 -0.71405103 27.42810207]


In [10]:
#Certovo aerobni
data = d[(d.Soil=='CT') & (d.Status=='A')]
data = data.reset_index(drop=True)
dataCmic = data.Cmic12init[0]

optimum_CA=differential_evolution(obj_fun, [(0, 1), (0.0001, 10), (0.1, 100), (0, 1), (0.0001, 10), (0.1, 100),
                                            (0.0001, 1), (0.0001, 10), (0, 1), 
                                            (1e-7, 1e-5), (0, 1), (0,1),
                                            (dataCmic*0.01, dataCmic*0.99)], 
                                  polish=True, maxiter=1000000)
print(optimum_CA)

KeyboardInterrupt: 

In [ ]:
print(goodness(optimum_CA.x))
np.savetxt('CT_parametersD.csv', optimum_CA.x.reshape(1,12), delimiter=",")